In [1]:
import pandas as pd
import os
import tiktoken
import openai
from scipy.spatial.distance import cosine
import os



In [5]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()

##  여기서부터 시작!!!!!!!!!!!
### 여기서부터 df=pd.read_csv("파일이름.csv") 로 내가 만든 데이터를 불러오기. 파일 열의 형식은 `title`, `content`이어야 한다.

In [ ]:
df = pd.read_csv(".././food_eng.csv")

In [ ]:
df

In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df['content'].apply(lambda x: len(tokenizer.encode(x)))
df.n_tokens.hist()

In [ ]:
max_tokens = 2000
def split_into_many(text, max_tokens = max_tokens):

    # 일단 문장으로 쪼개기
    sentences = text.split('. ')

    # 문장마다 몇 토큰 들어가는지 세어 주기.
    n_tokens = [len(tokenizer.encode(" " + sentence)) for sentence in sentences]
    
    chunks = []
    tokens_so_far = 0
    chunk = []


    for sentence, token in zip(sentences, n_tokens):

        # 문장의 토큰 수와 현재까지의 토큰 수의 합이 최대 토큰 수를 초과하면, 해당 청크를 청크 목록에 추가하고 청크 및 현재까지의 토큰 수를 초기화한다.
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + ".")
            chunk = []
            tokens_so_far = 0

        if token > max_tokens:
            continue

        chunk.append(sentence)
        tokens_so_far += token + 1

    if chunk:
        chunks.append(". ".join(chunk) + ".")

    return chunks

In [ ]:
shortened = []
# 데이터 프레임 순회하기
for row in df.iterrows():

    # If the content is None, go to the next row
    if row[1]['content'] is None:
        continue

    # If the number of tokens is greater than the max number of tokens, split the content into chunks
    if row[1]['n_tokens'] > max_tokens:
        shortened += split_into_many(row[1]['content'])
    
    # Otherwise, add the content to the list of shortened texts
    else:
        shortened.append( row[1]['content'] )

In [ ]:
def split_large_text(large_text, max_tokens=2000):
    enc = tiktoken.get_encoding("cl100k_base")
    tokenized_text = enc.encode(large_text)

    chunks = []
    current_chunk = []
    current_length = 0

    for token in tokenized_text:
        current_chunk.append(token)
        current_length += 1

        if current_length >= max_tokens:
            chunks.append(enc.decode(current_chunk).rstrip(' .,;'))
            current_chunk = []
            current_length = 0

    if current_chunk:
        chunks.append(enc.decode(current_chunk).rstrip(' .,;'))

    return chunks

df['splitted_content'] = df['content'].apply(split_large_text)
df = df.explode("splitted_content")
df['spllitted_n_tokens'] = df['splitted_content'].apply(lambda x: len(tokenizer.encode(x)))
df = df.reset_index( drop=True)

In [ ]:
df = df[['title','splitted_content', 'spllitted_n_tokens']].rename(columns={'splitted_content':'content', 'spllitted_n_tokens':'n_tokens'})

In [ ]:
def combine_info(row):
    title = " ".join(row['title'].split("-"))

    content = row['content']   #챗gpt 물어봐서 추가한 코드
    
    combined = [f"title: {title}", f"content: \n{content}"]
    combined = "\n\n###\n\n".join(combined)
    
    return combined

df["combined"] = df.apply(combine_info, axis=1)

In [ ]:
df

In [ ]:
# def get_embedding(text, model="text-embedding-ada-002"):
#    text = text.replace("\n", " ")
#    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
# df["embedding"] = df.combined.apply(get_embedding)
# df.to_pickle("../../processed/food_eng.pkl")

In [ ]:
df = pd.read_pickle("../../processed/food_eng.pkl")

##  여기서부터는 테스트 코드!!!!!!!!!!!!!!!!!!!
### 실제 개발환경에서 배포할 때 확인할 목적!

In [3]:
df = pd.read_pickle("../../processed/food_eng.pkl")
def create_context(
    question, df, max_len=3000
):
    q_embeddings = client.embeddings.create(input=question, model='text-embedding-ada-002').data[0].embedding
    df["distances"] = df["embedding"].apply(lambda x: cosine(q_embeddings, x))
    returns = []
    cur_len = 0
    for i, row in df.sort_values('distances', ascending=True).iterrows():
        cur_len += row['n_tokens'] + 4
        if cur_len > max_len:
            break
        returns.append(row["combined"])
    return "\n\n===\n\n".join(returns)


def answer_question(
    df,
    model="gpt-3.5-turbo",
    question="Tell me about Tumbler Beer",
    max_len=3000,
    debug=False,
):
    context = create_context(
        question,
        df,
        max_len=max_len,
    )
    if debug:
        print("Context:\n" + context)
        print("\n\n")

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\n"},
                {"role": "user", "content": f"context: {context}\n\n---\n\n Question: {question}, 한국어로 대답해줘."}
            ],
            temperature=0,
        )
        return response.choices[0].message.content
    except Exception as e:
        print("Error occurred:", e)
        return ""

In [6]:
answer_question(df, question="Tumbler Beer의 주소를 알려줘")

'Tumbler Beer의 주소는 서울 강남구 테헤란로 20길 15입니다.'

In [7]:
answer_question(df, question="Tumbler Beer의 운영 시간를 알려줘")

'Tumbler Beer의 운영 시간은 주중에는 오후 4시부터 새벽 3시까지이고, 토요일에는 오후 4시부터 밤 12시까지입니다.'

In [8]:
answer_question(df, question="Tumbler Beer의 교통 정보를 알려줘")

'지하철 2호선 역삼역 3번 출구에서 이동할 수 있습니다.'